In [8]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

In [9]:
# Set the file path
filepath = Path("../Resources/aapl_data.csv")

# Read the CSV located at the file path into a Pandas DataFrame
# Set the `Date` column as the index and auto-format the datetime string
aapl_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
aapl_df.head()

,Date,Open,High,Low,Close,Volume,Volatility,sentiment,RSI
0,2019-01-02,154.89,158.8500,154.23,157.5225,27803621,0.214265,-0.048624,NaN
1,2019-01-03,143.98,145.7200,142.00,142.2900,82418448,0.407750,-0.037484,NaN
2,2019-01-04,144.53,148.5499,143.80,148.1200,51267225,0.538786,0.005249,NaN
3,2019-01-07,148.70,148.8300,145.90,147.9100,49982480,0.115592,0.012711,NaN
4,2019-01-08,149.56,151.8200,148.52,150.6300,36592406,0.114619,0.019823,NaN


In [14]:
# Grab just the `date` and `close` from the IEX dataset
signals_df = aapl_df.loc[:, ["Date", "Close"]].copy()

# Set the short window and long windows
short_window = 5
long_window = 15

# Set the `date` column as the index
signals_df = signals_df.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

signals_df.head(10)

,Close,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,
2019-01-02,157.5225,NaN,NaN,0.0,NaN
2019-01-03,142.2900,NaN,NaN,0.0,0.0
2019-01-04,148.1200,NaN,NaN,0.0,0.0
2019-01-07,147.9100,NaN,NaN,0.0,0.0
2019-01-08,150.6300,149.29450,NaN,0.0,0.0
2019-01-09,153.3700,148.46400,NaN,0.0,0.0
2019-01-10,153.5500,150.71600,NaN,0.0,0.0
2019-01-11,152.0534,151.50268,NaN,0.0,0.0
2019-01-14,149.7600,151.87268,NaN,0.0,0.0


In [15]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)